In [1]:
import numpy

from cicada.additive import AdditiveProtocolSuite
from cicada.communicator import SocketCommunicator
from cicada.encoding import FixedPoint

def main(communicator):
    encoding = FixedPoint(precision=2)
    protocol = AdditiveProtocolSuite(communicator, order=127, encoding=encoding)
    a_share = protocol.share(src=0, secret=numpy.array(2), shape=())
    b_share = protocol.share(src=1, secret=numpy.array(3), shape=())
    c_share = protocol.add(a_share, b_share)
    c = protocol.reveal(c_share)

Note that we're using a very low-precision encoding (`precision=2`) and a very small field (`order=127`) to make the values in the logged output easier to read.

In [2]:
import logging

from cicada import transcript

with transcript.record():
    handler = transcript.net_handler()
    transcript.set_handler(logging.getLogger(), handler)
    SocketCommunicator.run(fn=main, world_size=3);

Player 2: --> 0 PRZS 1838865260088296811
Player 0: --> 1 PRZS 5186513781199103102
Player 1: --> 2 PRZS 1109593088794431382
Player 0: <-- 2 PRZS 1838865260088296811
Player 2: <-- 1 PRZS 1109593088794431382
Player 1: <-- 0 PRZS 5186513781199103102
Player 2: --> 0 GATHER 113
Player 0: --> 0 GATHER 69
Player 1: --> 0 GATHER 92
Player 2: --> 1 GATHER 113
Player 0: <-- 0 GATHER 69
Player 1: --> 1 GATHER 92
Player 2: --> 2 GATHER 113
Player 0: <-- 2 GATHER 113
Player 1: <-- 2 GATHER 113
Player 0: <-- 1 GATHER 92
Player 2: <-- 2 GATHER 113
Player 1: <-- 1 GATHER 92
Player 0: --> 1 GATHER 69
Player 0: --> 2 GATHER 69
Player 1: <-- 0 GATHER 69
Player 2: <-- 0 GATHER 69
Player 1: --> 2 GATHER 92
Player 2: <-- 1 GATHER 92


In [3]:
def main(communicator):
    transcript.log("Let's setup additive sharing!")
    encoding = FixedPoint(precision=2)
    protocol = AdditiveProtocolSuite(communicator, order=127, encoding=encoding)
    transcript.log("Let's share some secrets!")
    a_share = protocol.share(src=0, secret=numpy.array(2), shape=())
    b_share = protocol.share(src=1, secret=numpy.array(3), shape=())
    transcript.log("Let's add some secrets!")
    c_share = protocol.add(a_share, b_share)
    transcript.log("Let's reveal the results!")
    c = protocol.reveal(c_share)

with transcript.record():
    handler = transcript.net_handler()
    transcript.set_handler(logging.getLogger(), handler)
    SocketCommunicator.run(fn=main, world_size=3);

Player 0: Let's setup additive sharing!
Player 2: Let's setup additive sharing!
Player 1: Let's setup additive sharing!
Player 0: --> 1 PRZS 8589156650860222972
Player 1: --> 2 PRZS 6145510639690000126
Player 2: --> 0 PRZS 4437933377607861447
Player 1: <-- 0 PRZS 8589156650860222972
Player 2: <-- 1 PRZS 6145510639690000126
Player 0: <-- 2 PRZS 4437933377607861447
Player 1: Let's share some secrets!
Player 2: Let's share some secrets!
Player 0: Let's share some secrets!
Player 1: Let's add some secrets!
Player 2: Let's add some secrets!
Player 0: Let's add some secrets!
Player 1: Let's reveal the results!
Player 2: Let's reveal the results!
Player 0: Let's reveal the results!
Player 1: --> 0 GATHER 102
Player 2: --> 0 GATHER 51
Player 0: --> 0 GATHER 121
Player 1: --> 1 GATHER 102
Player 2: --> 1 GATHER 51
Player 0: <-- 1 GATHER 102
Player 2: --> 2 GATHER 51
Player 0: <-- 2 GATHER 51
Player 1: <-- 1 GATHER 102
Player 1: <-- 2 GATHER 51
Player 2: <-- 2 GATHER 51
Player 0: <-- 0 GATHER 12

In [4]:
with transcript.record():
    fmt = "# {processName}: {msg}"
    netfmt = "{processName},{net.verb},{net.src},{net.dst},{net.tag},{net.payload}"
    handler = transcript.net_handler(fmt=fmt, netfmt=netfmt)
    transcript.set_handler(logging.getLogger(), handler)
    SocketCommunicator.run(fn=main, world_size=3);

# Player 0: Let's setup additive sharing!
# Player 1: Let's setup additive sharing!
# Player 2: Let's setup additive sharing!
Player 0,sent,0,1,PRZS,5249556167448130605
Player 2,sent,2,0,PRZS,746859899813256019
Player 1,sent,1,2,PRZS,5725786111065523107
Player 0,received,2,0,PRZS,746859899813256019
Player 1,received,0,1,PRZS,5249556167448130605
Player 2,received,1,2,PRZS,5725786111065523107
# Player 0: Let's share some secrets!
# Player 1: Let's share some secrets!
# Player 2: Let's share some secrets!
# Player 0: Let's add some secrets!
# Player 2: Let's add some secrets!
# Player 1: Let's add some secrets!
# Player 0: Let's reveal the results!
# Player 2: Let's reveal the results!
# Player 1: Let's reveal the results!
Player 0,sent,0,0,GATHER,43
Player 2,sent,2,0,GATHER,6
Player 1,sent,1,0,GATHER,98
Player 0,received,0,0,GATHER,43
Player 2,sent,2,1,GATHER,6
Player 1,sent,1,1,GATHER,98
Player 0,received,2,0,GATHER,6
Player 2,sent,2,2,GATHER,6
Player 1,received,2,1,GATHER,6
Player 0,re

If you look carefully, you can see that this produces two nearly identical events for each message (once when the message is sent, and once when the message is received).  If you wish to eliminate the duplication, e.g. by only logging messages when they're sent, you can specify that too:

In [5]:
with transcript.record():
    fmt = "# {processName}: {msg}"
    netfmt = "{processName},{net.verb},{net.src},{net.dst},{net.tag},{net.payload}"
    handler = transcript.net_handler(received=False, fmt=fmt, netfmt=netfmt)
    transcript.set_handler(logging.getLogger(), handler)
    SocketCommunicator.run(fn=main, world_size=3);

# Player 0: Let's setup additive sharing!
# Player 1: Let's setup additive sharing!
# Player 2: Let's setup additive sharing!
Player 2,sent,2,0,PRZS,7773395554418552545
Player 0,sent,0,1,PRZS,3880388577136065605
Player 1,sent,1,2,PRZS,4808656572318633561
# Player 2: Let's share some secrets!
# Player 0: Let's share some secrets!
# Player 1: Let's share some secrets!
# Player 2: Let's add some secrets!
# Player 0: Let's add some secrets!
# Player 1: Let's add some secrets!
# Player 2: Let's reveal the results!
# Player 0: Let's reveal the results!
# Player 1: Let's reveal the results!
Player 2,sent,2,0,GATHER,77
Player 0,sent,0,0,GATHER,77
Player 1,sent,1,0,GATHER,120
Player 2,sent,2,1,GATHER,77
Player 1,sent,1,1,GATHER,120
Player 2,sent,2,2,GATHER,77
Player 0,sent,0,1,GATHER,77
Player 0,sent,0,2,GATHER,77
Player 1,sent,1,2,GATHER,120


In [6]:
from cicada.arithmetic import Field

with transcript.record():
    handler = transcript.code_handler()
    transcript.set_handler(logging.getLogger(), handler)
    
    f = Field(order=127)
    a = f.ones(3)
    b = f.uniform(size=3, generator=numpy.random.default_rng())
    f.inplace_add(a, b)

cicada.transcript.assert_equal(cicada.arithmetic.Field(order=127).ones(shape=3), numpy.array([1, 1, 1], dtype='object'))

bg = numpy.random.PCG64()
bg.state = {'bit_generator': 'PCG64', 'state': {'state': 105482563193143646269675226417058719126, 'inc': 127921216801897068283996638341130724407}, 'has_uint32': 0, 'uinteger': 0}
cicada.transcript.assert_equal(cicada.arithmetic.Field(order=127).uniform(size=3, generator=numpy.random.Generator(bg)), numpy.array([55, 21, 24], dtype='object'))

lhs = numpy.array([1, 1, 1], dtype='object')
cicada.arithmetic.Field(order=127).inplace_add(lhs=lhs, rhs=numpy.array([55, 21, 24], dtype='object'))
cicada.transcript.assert_equal(lhs, numpy.array([56, 22, 25], dtype='object'))



In [7]:
import io

with transcript.record():
    buffer = io.StringIO()
    handler = transcript.code_handler(logging.StreamHandler(buffer))
    transcript.set_handler(logging.getLogger(), handler)
    
    f = Field(order=127)
    a = f.ones(3)
    b = f.uniform(size=3, generator=numpy.random.default_rng())
    f.inplace_add(a, b)

In [8]:
buffer.seek(0)
for line in buffer:
    print(line.strip())

cicada.transcript.assert_equal(cicada.arithmetic.Field(order=127).ones(shape=3), numpy.array([1, 1, 1], dtype='object'))

bg = numpy.random.PCG64()
bg.state = {'bit_generator': 'PCG64', 'state': {'state': 223201357120696992247529150820340694747, 'inc': 14766786733188267275985428446609739979}, 'has_uint32': 0, 'uinteger': 0}
cicada.transcript.assert_equal(cicada.arithmetic.Field(order=127).uniform(size=3, generator=numpy.random.Generator(bg)), numpy.array([93, 64, 28], dtype='object'))

lhs = numpy.array([1, 1, 1], dtype='object')
cicada.arithmetic.Field(order=127).inplace_add(lhs=lhs, rhs=numpy.array([93, 64, 28], dtype='object'))
cicada.transcript.assert_equal(lhs, numpy.array([94, 65, 29], dtype='object'))



Now, we can execute the buffered transcript, one-line-at-a-time, and confirm that none of the assertions raise exceptions:

In [9]:
import cicada.transcript

try:
    buffer.seek(0)
    for line in buffer:
        exec(line)
except Exception as e:
    print(f"Consistency verification failed: {e}")
else:
    print("Consistency verification succeeded.")

Consistency verification succeeded.


Let's go back to our original example. Because it requires three players, we'll setup Cicada to write a separate transcript for each player:  

In [10]:
def main(comm):
    handler = transcript.code_handler(logging.FileHandler(f"player-{comm.rank}.py", "w"))
    transcript.set_handler(logging.getLogger(), handler)

    transcript.log("Let's setup additive sharing!")
    encoding = FixedPoint(precision=2)
    protocol = AdditiveProtocolSuite(comm, order=127, encoding=encoding)
    transcript.log("Let's share some secrets!")
    a_share = protocol.share(src=0, secret=numpy.array(2), shape=())
    b_share = protocol.share(src=1, secret=numpy.array(3), shape=())
    transcript.log("Let's add some secrets!")
    c_share = protocol.add(a_share, b_share)
    transcript.log("Let's reveal the results!")
    c = protocol.reveal(c_share)

with transcript.record():
    SocketCommunicator.run(fn=main, world_size=3);

Note that we're configuring the transcript logging inside `main`, since that allows us to generate a separate filename for each trasncript based on the players' rank. Also note that we've left our context messages in-place. Let's see what the results look like for one of the players:

In [11]:
with open("player-0.py") as stream:
    for line in stream:
        print(line.strip())

# Let's setup additive sharing!
# Let's share some secrets!
# cicada.additive.AdditiveProtocolSuite().share(src=0, secret=numpy.array(2, dtype='int64'), shape=(), encoding=None)

bg = numpy.random.PCG64()
bg.state = {'bit_generator': 'PCG64', 'state': {'state': 52757503511119475181362121394601717382, 'inc': 333705614055274107041924419032548720619}, 'has_uint32': 0, 'uinteger': 0}
cicada.transcript.assert_equal(cicada.arithmetic.Field(order=127).uniform(size=(), generator=numpy.random.Generator(bg)), numpy.array(124, dtype='object'))

bg = numpy.random.PCG64()
bg.state = {'bit_generator': 'PCG64', 'state': {'state': 15228111457475569490721819116347463097, 'inc': 200986557084021399194868578718906373973}, 'has_uint32': 0, 'uinteger': 0}
cicada.transcript.assert_equal(cicada.arithmetic.Field(order=127).uniform(size=(), generator=numpy.random.Generator(bg)), numpy.array(0, dtype='object'))

lhs = numpy.array(124, dtype='object')
cicada.arithmetic.Field(order=127).inplace_subtract(lhs=lhs, r

In [12]:
import traceback

for rank in range(3):
    try:
        with open(f"player-{rank}.py") as stream:
            for line in stream:
                exec(line)
    except Exception as e:
        print(f"Player {rank} consistency verification failed: {e}")
        print(traceback.format_exc())
    else:
        print(f"Player {rank} consistency verification succeeded.")

Player 0 consistency verification succeeded.
Player 1 consistency verification succeeded.
Player 2 consistency verification succeeded.


In [13]:
def main(comm):
    handler = logging.StreamHandler() if comm.rank == 0 else logging.NullHandler()
    handler = transcript.code_handler(handler, sent=True, received=True)
    transcript.set_handler(logging.getLogger(), handler)

    transcript.log("Let's setup additive sharing!")
    encoding = FixedPoint(precision=2)
    protocol = AdditiveProtocolSuite(comm, order=127, encoding=encoding)
    transcript.log("Let's share some secrets!")
    a_share = protocol.share(src=0, secret=numpy.array(2), shape=())
    b_share = protocol.share(src=1, secret=numpy.array(3), shape=())
    transcript.log("Let's add some secrets!")
    c_share = protocol.add(a_share, b_share)
    transcript.log("Let's reveal the results!")
    c = protocol.reveal(c_share)

with transcript.record():
    SocketCommunicator.run(fn=main, world_size=3);

# Let's setup additive sharing!
# 0 --> 1 PRZS 2209658626268423143
# 0 <-- 2 PRZS 8998672946641975369
# 0 <-- 2 GATHER 20
# Let's share some secrets!
# 0 <-- 1 GATHER 88
# cicada.additive.AdditiveProtocolSuite().share(src=0, secret=numpy.array(2, dtype='int64'), shape=(), encoding=None)

bg = numpy.random.PCG64()
bg.state = {'bit_generator': 'PCG64', 'state': {'state': 261246562912492245002531388094879914742, 'inc': 14944546162778457561862776155980357503}, 'has_uint32': 0, 'uinteger': 0}
cicada.transcript.assert_equal(cicada.arithmetic.Field(order=127).uniform(size=(), generator=numpy.random.Generator(bg)), numpy.array(27, dtype='object'))

bg = numpy.random.PCG64()
bg.state = {'bit_generator': 'PCG64', 'state': {'state': 293659148647216836055213682163626256711, 'inc': 276545542395767621607894736996487803825}, 'has_uint32': 0, 'uinteger': 0}
cicada.transcript.assert_equal(cicada.arithmetic.Field(order=127).uniform(size=(), generator=numpy.random.Generator(bg)), numpy.array(113, dtype='